In [1]:
import pandas as pd
import datetime as dt
import hvplot.pandas

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
coin_list = cg.get_coins_list()

coins = pd.DataFrame(coin_list)

coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0vix-protocol,vix,0VIX Protocol
4,0x,zrx,0x Protocol
...,...,...,...
10041,zynecoin,zyn,Zynecoin
10042,zynergy,zyn,Zynergy
10043,zyrri,zyr,Zyrri
10044,zyx,zyx,ZYX


In [4]:
tokens = [
    'GTC',
    'GIV',
    'ENS',
    'KLIMA',
    'RAD',
    'BRIGHT',
    'HONEY',
    'TEC',
    'SEED',
    'PAN',
]

In [5]:
coins = coins[coins['symbol'].str.contains('|'.join(tokens).lower())]
coins

,id,symbol,name
545,arbpanda-ai,aipanda,ArbPanda AI
669,astra-dao,astradao,Astra DAO
744,ausd-seed-acala,aseed,aUSD SEED (Acala)
745,ausd-seed-karura,aseed,aUSD SEED (Karura)
1078,big-panda,panda,Big Panda
...,...,...,...
8744,token-engineering-commons,tec,Token Engineering Commons
8808,toxicgarden-finance-seed,seed,ToxicGarden.finance SEED
8826,tradex-ai,tradex,TradeX AI
8914,tupan,tupan,Tupan


In [6]:
token_ids = [
    'bright-token',
    # 'ethereum-name-service',
    'gitcoin',
    'giveth',
    'rad',
    'honey-finance',
    'token-engineering-commons',
    'metagame',
    
]

exact_match_regex = ['^'+id+'$' for id in token_ids]

In [7]:
coins[coins['id'].str.match('|'.join(exact_match_regex).lower())]

,id,symbol,name
1463,bright-token,bright,BrightID
3688,gitcoin,gtc,Gitcoin
3692,giveth,giv,Giveth
4129,honey-finance,honey,Honey Finance
5473,metagame,seed,MetaGame
7174,rad,rad,RAD
8744,token-engineering-commons,tec,Token Engineering Commons


In [8]:
token_id = 'bright-token'
quote = 'ETH'
days = 730

# Pull prices from coin gecko
prices = cg.get_coin_market_chart_by_id(token_id, quote, days)

# Little bit of clean up
prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
prices = {a:[i[1] for i in b] for a,b in prices.items()}

# Make a dataframe time series
prices_df = pd.DataFrame(prices)

prices_df['token_id'] = token_id

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000322,0.0,80.248617,1632096000000,bright-token
1,0.000270,0.0,92.635804,1632182400000,bright-token
2,0.000268,0.0,50.559488,1632268800000,bright-token
3,0.000254,0.0,46.795054,1632355200000,bright-token
4,0.000233,0.0,41.993178,1632441600000,bright-token
...,...,...,...,...,...
726,0.000021,0.0,0.053504,1694822400000,bright-token
727,0.000022,0.0,4.189035,1694908800000,bright-token
728,0.000022,0.0,0.095688,1694995200000,bright-token
729,0.000021,0.0,0.129893,1695081600000,bright-token


In [9]:
def get_price_history(token_id: str, quote: str, days: int) -> pd.DataFrame:
    # Pull prices from coin gecko
    prices = cg.get_coin_market_chart_by_id(token_id, quote, days)
    
    # Little bit of clean up
    prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
    prices = {a:[i[1] for i in b] for a,b in prices.items()}
    
    # Make a dataframe time series
    prices_df = pd.DataFrame(prices)
    
    prices_df['token_id'] = token_id
    
    return prices_df

In [10]:
token_ids

['bright-token',
 'gitcoin',
 'giveth',
 'rad',
 'honey-finance',
 'token-engineering-commons',
 'metagame']

In [11]:
df = get_price_history(token_ids[1], quote='ETH', days=730)
df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.002607,37000.152189,3107.208435,1632096000000,gitcoin
1,0.002451,34597.926415,5947.694940,1632182400000,gitcoin
2,0.002481,35306.942321,4790.914111,1632268800000,gitcoin
3,0.002575,36720.182707,3522.080068,1632355200000,gitcoin
4,0.002610,37094.495294,3388.123392,1632441600000,gitcoin
...,...,...,...,...,...
726,0.000561,34302.175322,2533.209276,1694822400000,gitcoin
727,0.000568,34346.413448,2838.454852,1694908800000,gitcoin
728,0.000545,33133.303592,4506.285936,1694995200000,gitcoin
729,0.000545,33213.900903,2625.051774,1695081600000,gitcoin


In [12]:
data = []
for token_id in token_ids:
    print(f"Fetching data for token_id: {token_id}")
    df = get_price_history(token_id, quote='ETH', days=730)
    data.append(df)

print(f"Fetching data for ETH/USD")
df = get_price_history('ethereum', quote='usd', days=730)
data.append(df)

Fetching data for token_id: bright-token
Fetching data for token_id: gitcoin
Fetching data for token_id: giveth
Fetching data for token_id: rad
Fetching data for token_id: honey-finance
Fetching data for token_id: token-engineering-commons
Fetching data for token_id: metagame
Fetching data for ETH/USD


In [13]:
# Concatenate all of the data into one dataframe
prices_df = pd.concat(data)

# Create datetime column from the timestamp
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'], unit='ms')

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token
...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum


In [14]:
import hvplot.pandas

In [15]:
prices_df.hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

Create normalized prices.

In [16]:
prices_df['normalized_prices'] = prices_df['prices'] / prices_df.groupby('token_id')['prices'].transform('first')

In [17]:
prices_df.hvplot.step(x='timestamp', y='normalized_prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (normalized_prices)

Giveth has some anomolous behavior near the beginning. I'm going to remove the first 20 days of giveth.

In [18]:
import holoviews as hv

prices_df_adjusted = prices_df.copy(deep=True)

prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'] = prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'].iloc[20:]

prices_df_adjusted['normalized_prices'] = prices_df_adjusted['prices'] / prices_df_adjusted.groupby('token_id')['prices'].transform('first')

In [19]:
import datetime as dt

In [20]:
d = dt.date.today()

In [21]:
import datetime

def ordinal(n):
    """Return number with ordinal suffix."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

def formatted_date(past=False):
    """Return today's date in the desired format."""
    today = datetime.date.today()
    month = today.strftime('%b')
    day = ordinal(today.day)
    year = today.year
    if past:
        year = year - 2
    return f"{month} {day} {year}"

print(formatted_date())

Sep 19th 2023


In [22]:
today = formatted_date()
past = formatted_date(past=True)

In [23]:
plot = prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(logy=True, legend='bottom_left', grid=True, rot=45, title=f'Public Goods Tokens Normalized against Ethereum {past} - {today}',x='timestamp', y='normalized_prices', by='token_id', width=1400, height=600, line_width=3) * hv.HLine(1)
plot

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [24]:
plot.opts(xticks=[(date, date.strftime('%B %Y')) for date in pd.date_range(start=prices_df_adjusted['timestamp'].min(), end=prices_df_adjusted['timestamp'].max(), freq='MS')])

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [25]:
prices_df_adjusted['returns'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x.bfill().pct_change()+1)

In [26]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,1.000000,NaN
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.838049,0.838049
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.833386,0.994436
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.789774,0.947669
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token,0.725248,0.918298
...,...,...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.492204,1.009529
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.489951,0.995423
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.486371,0.992694
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum,0.490615,1.008725


In [27]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
gitcoin,1.000093,0.079956,0.001158
giveth,0.999502,0.167165,-0.002981
bright-token,0.998942,0.076746,-0.013779
token-engineering-commons,0.998393,0.046729,-0.034395
honey-finance,0.997714,0.052273,-0.043741
metagame,0.996428,0.061730,-0.057873
rad,0.993804,0.087567,-0.070751


In [28]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)

token_id
token-engineering-commons    4.341073
honey-finance                2.736850
gitcoin                      2.616769
metagame                     1.677495
bright-token                 0.869051
giveth                       0.134364
rad                          0.047730
dtype: float64

In [29]:
prices_df_adjusted.groupby('token_id')['normalized_prices'].apply(lambda x: (x.iloc[-1] / x.std())).sort_values(ascending=False)

token_id
ethereum                     1.740646
token-engineering-commons    0.868541
gitcoin                      0.673812
bright-token                 0.458992
metagame                     0.420306
honey-finance                0.315044
giveth                       0.094325
rad                          0.017039
Name: normalized_prices, dtype: float64

In [30]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,1.000000,NaN
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.838049,0.838049
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.833386,0.994436
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.789774,0.947669
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token,0.725248,0.918298
...,...,...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.492204,1.009529
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.489951,0.995423
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.486371,0.992694
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum,0.490615,1.008725


In [31]:
prices_df_adjusted.reset_index()['returns'].bfill().idxmax()

1628

In [32]:
prices_df_adjusted.iloc[4484]

prices                       1214.866265
market_caps          146866764082.715027
total_volumes         32772246927.059067
timestamp            2022-06-15 00:00:00
token_id                        ethereum
normalized_prices               0.364181
returns                          1.00769
Name: 268, dtype: object

In [33]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,1.000000,NaN
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.838049,0.838049
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.833386,0.994436
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.789774,0.947669
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token,0.725248,0.918298
...,...,...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.492204,1.009529
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.489951,0.995423
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.486371,0.992694
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum,0.490615,1.008725


In [34]:
prices_df_adjusted[prices_df_adjusted['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.001200,0.0,4.412721,2022-01-25 00:00:00,token-engineering-commons,1.000000,NaN
1,0.001200,0.0,4.412721,2022-01-26 00:00:00,token-engineering-commons,1.000000,1.000000
2,0.000953,0.0,2.566614,2022-01-27 00:00:00,token-engineering-commons,0.794553,0.794553
3,0.001048,0.0,2.058658,2022-01-28 00:00:00,token-engineering-commons,0.873267,1.099067
4,0.000959,0.0,0.155690,2022-01-29 00:00:00,token-engineering-commons,0.798987,0.914940
...,...,...,...,...,...,...,...
598,0.000251,0.0,0.072697,2023-09-16 00:00:00,token-engineering-commons,0.209519,0.970346
599,0.000246,0.0,0.095854,2023-09-17 00:00:00,token-engineering-commons,0.205332,0.980016
600,0.000263,0.0,0.008110,2023-09-18 00:00:00,token-engineering-commons,0.219207,1.067573
601,0.000243,0.0,0.001919,2023-09-18 20:30:23,token-engineering-commons,0.202855,0.925406


In [35]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last').sort_values(ascending=False).to_frame(name=f'Return {past} - {today}').reset_index()

,token_id,Return Sep 19th 2021 - Sep 19th 2023
0,gitcoin,0.209227
1,token-engineering-commons,0.202855
2,honey-finance,0.143064
3,metagame,0.103552
4,bright-token,0.066696
5,giveth,0.022461
6,rad,0.004180


In [36]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std').sort_values(ascending=True).to_frame(name=f'Volatility {past} - {today}').reset_index()

,token_id,Volatility Sep 19th 2021 - Sep 19th 2023
0,token-engineering-commons,0.046729
1,honey-finance,0.052273
2,metagame,0.061730
3,bright-token,0.076746
4,gitcoin,0.079956
5,rad,0.087567
6,giveth,0.167165


In [37]:
(prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last') / prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std')).sort_values(ascending=False).to_frame(name=f'Pseudo Sharpe {past} - {today}').reset_index()

,token_id,Pseudo Sharpe Sep 19th 2021 - Sep 19th 2023
0,token-engineering-commons,4.341073
1,honey-finance,2.736850
2,gitcoin,2.616769
3,metagame,1.677495
4,bright-token,0.869051
5,giveth,0.134364
6,rad,0.047730


In [38]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
gitcoin,1.000093,0.079956,0.001158
giveth,0.999502,0.167165,-0.002981
bright-token,0.998942,0.076746,-0.013779
token-engineering-commons,0.998393,0.046729,-0.034395
honey-finance,0.997714,0.052273,-0.043741
metagame,0.996428,0.061730,-0.057873
rad,0.993804,0.087567,-0.070751


In [39]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,1.000000,NaN
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.838049,0.838049
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.833386,0.994436
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.789774,0.947669
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token,0.725248,0.918298
...,...,...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.492204,1.009529
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.489951,0.995423
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.486371,0.992694
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum,0.490615,1.008725


In [40]:
prices_df_adjusted['returns'].rolling(7).std()

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
726    0.022316
727    0.022401
728    0.022188
729    0.011924
730    0.007406
Name: returns, Length: 4947, dtype: float64

In [41]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,1.000000,NaN
1,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.838049,0.838049
2,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.833386,0.994436
3,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.789774,0.947669
4,0.000233,0.000000e+00,4.199318e+01,2021-09-24 00:00:00,bright-token,0.725248,0.918298
...,...,...,...,...,...,...,...
726,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.492204,1.009529
727,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.489951,0.995423
728,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.486371,0.992694
729,1636.634609,1.968665e+11,9.941459e+09,2023-09-19 00:00:00,ethereum,0.490615,1.008725


In [42]:
prices_df_adjusted_tokens = prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].copy(deep=True)

In [43]:
prices_df_adjusted_tokens['volatility'] = prices_df_adjusted_tokens.groupby('token_id')['returns'].transform(
    lambda x: x.rolling(90).std().bfill()
)

In [44]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000322,0.0,80.248617,2021-09-20 00:00:00,bright-token,1.000000,NaN,0.181025
1,0.000270,0.0,92.635804,2021-09-21 00:00:00,bright-token,0.838049,0.838049,0.181025
2,0.000268,0.0,50.559488,2021-09-22 00:00:00,bright-token,0.833386,0.994436,0.181025
3,0.000254,0.0,46.795054,2021-09-23 00:00:00,bright-token,0.789774,0.947669,0.181025
4,0.000233,0.0,41.993178,2021-09-24 00:00:00,bright-token,0.725248,0.918298,0.181025
...,...,...,...,...,...,...,...,...
404,0.000954,0.0,0.009536,2023-09-15 00:00:00,metagame,0.104082,0.999903,0.057138
405,0.000947,0.0,0.094714,2023-09-17 00:00:00,metagame,0.103380,0.993259,0.057136
406,0.000947,0.0,0.094714,2023-09-18 00:00:00,metagame,0.103380,1.000000,0.056824
407,0.000949,0.0,0.094871,2023-09-18 06:20:01,metagame,0.103552,1.001661,0.056825


In [45]:
prices_df_adjusted_tokens.hvplot.line(
    logy=True, 
    legend='top_right', 
    grid=True, rot=45, 
    title=f'Public Goods Tokens 90 Day Volatility Against Ethereum {past} - {today}',
    x='timestamp', y='volatility', by='token_id', width=1400, height=600, line_width=3) 

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (volatility)

In [46]:
prices_df_adjusted_tokens

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns,volatility
0,0.000322,0.0,80.248617,2021-09-20 00:00:00,bright-token,1.000000,NaN,0.181025
1,0.000270,0.0,92.635804,2021-09-21 00:00:00,bright-token,0.838049,0.838049,0.181025
2,0.000268,0.0,50.559488,2021-09-22 00:00:00,bright-token,0.833386,0.994436,0.181025
3,0.000254,0.0,46.795054,2021-09-23 00:00:00,bright-token,0.789774,0.947669,0.181025
4,0.000233,0.0,41.993178,2021-09-24 00:00:00,bright-token,0.725248,0.918298,0.181025
...,...,...,...,...,...,...,...,...
404,0.000954,0.0,0.009536,2023-09-15 00:00:00,metagame,0.104082,0.999903,0.057138
405,0.000947,0.0,0.094714,2023-09-17 00:00:00,metagame,0.103380,0.993259,0.057136
406,0.000947,0.0,0.094714,2023-09-18 00:00:00,metagame,0.103380,1.000000,0.056824
407,0.000949,0.0,0.094871,2023-09-18 06:20:01,metagame,0.103552,1.001661,0.056825


In [47]:
df = prices_df_adjusted_tokens.pivot_table(index='timestamp', columns='token_id', values='returns').resample('D').mean()
df

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
timestamp,,,,,,,
2021-09-21,0.838049,0.940366,NaN,NaN,0.962710,NaN,NaN
2021-09-22,0.994436,1.011957,NaN,NaN,0.997479,NaN,NaN
2021-09-23,0.947669,1.037981,NaN,NaN,1.077453,NaN,NaN
2021-09-24,0.918298,1.013723,NaN,NaN,0.899321,NaN,NaN
2021-09-25,0.954152,0.919384,NaN,NaN,0.954655,NaN,NaN
...,...,...,...,...,...,...,...
2023-09-15,0.994377,0.970169,0.979220,0.978519,0.999903,NaN,0.979974
2023-09-16,0.973879,1.008558,0.983139,0.995331,NaN,NaN,0.970346
2023-09-17,1.011821,1.013240,1.012888,1.001825,0.993259,NaN,0.980016


In [48]:
df.isna().sum()

token_id
bright-token                   0
gitcoin                        0
giveth                       113
honey-finance                196
metagame                     322
rad                          162
token-engineering-commons    128
dtype: int64

In [49]:
df.cumprod().bfill().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [50]:
cov = df.corr()
cov

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
token_id,,,,,,,
bright-token,1.000000,0.002675,0.014806,0.152713,-0.017008,0.089421,0.164161
gitcoin,0.002675,1.000000,-0.034541,-0.099585,0.011309,-0.019883,-0.018864
giveth,0.014806,-0.034541,1.000000,0.085772,0.336784,0.025879,0.053998
honey-finance,0.152713,-0.099585,0.085772,1.000000,0.174451,0.449519,0.272246
metagame,-0.017008,0.011309,0.336784,0.174451,1.000000,0.083767,0.037714
rad,0.089421,-0.019883,0.025879,0.449519,0.083767,1.000000,0.069458
token-engineering-commons,0.164161,-0.018864,0.053998,0.272246,0.037714,0.069458,1.000000


In [51]:
cov.hvplot.heatmap(cmap='reds', colorbar=True, rot=45, width=800, height=500, title="Public Goods Tokens Correlation")

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


:HeatMap   [columns,index]   (value)

In [52]:
df.mean()

token_id
bright-token                 0.998944
gitcoin                      1.000092
giveth                       0.999487
honey-finance                0.997712
metagame                     0.996417
rad                          0.993810
token-engineering-commons    0.998396
dtype: float64

# This section needs to be corrected.

Don't impute. Don't dropna. 

Start with prices, backfill missing data. Then compute returns.

In [53]:
import numpy as np

In [54]:
df_filled = df.where(~df.isna(), np.random.normal(df.mean(), df.std(), size=df.shape))

In [55]:
df_filled = df.dropna()

In [56]:
df_filled.cumprod().hvplot.line(logy=True)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (value)

In [57]:
df_filled

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
timestamp,,,,,,,
2022-04-09,0.949924,0.927148,1.002523,1.041729,1.145661,0.987208,1.016271
2022-04-10,0.968138,1.038673,0.976691,0.928047,1.000000,0.783287,0.958001
2022-04-11,0.984979,0.976918,1.060522,1.225002,0.998845,0.910245,1.019078
2022-04-12,1.110989,0.943426,0.985020,1.091937,1.110623,1.009557,1.010602
2022-04-13,0.889215,1.046814,0.937338,0.955503,0.999818,0.936462,0.984746
...,...,...,...,...,...,...,...
2023-08-27,1.004554,1.006104,1.001380,0.999220,1.037751,0.999940,1.007593
2023-08-30,0.980725,1.011458,1.001554,0.940267,0.926243,1.000000,1.005719
2023-08-31,1.021863,1.009520,0.995744,1.019316,1.000000,1.012279,0.987711


In [58]:
N = 1000
portfolios = pd.DataFrame(np.random.rand(N, df_filled.shape[1]), columns=df_filled.columns)
portfolios = (portfolios.T / portfolios.sum(axis=1)).T
portfolios

token_id,bright-token,gitcoin,giveth,honey-finance,metagame,rad,token-engineering-commons
0,0.185771,0.119075,0.170145,0.181107,0.061020,0.156834,0.126048
1,0.006939,0.098863,0.192648,0.143152,0.215732,0.218808,0.123857
2,0.203985,0.143315,0.160638,0.100500,0.016719,0.113452,0.261391
3,0.056851,0.111578,0.148568,0.321547,0.078670,0.260516,0.022271
4,0.132786,0.125444,0.083051,0.001276,0.280735,0.352359,0.024350
...,...,...,...,...,...,...,...
995,0.042669,0.339043,0.152258,0.070724,0.282065,0.069156,0.044086
996,0.213628,0.167382,0.122346,0.031516,0.057439,0.254686,0.153003
997,0.008503,0.043005,0.085524,0.236746,0.296528,0.066076,0.263619
998,0.162285,0.151400,0.153289,0.176174,0.168050,0.043055,0.145747


In [59]:
outcomes = df_filled @ portfolios.T
outcomes

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-04-09,0.998942,1.029550,0.989181,1.011306,1.021255,1.010785,1.009774,0.976105,1.021815,1.036504,...,1.000696,1.036440,1.035610,1.009003,0.998134,1.017417,0.986330,1.053173,1.014881,1.011249
2022-04-10,0.942407,0.936191,0.952503,0.918512,0.921210,0.915845,0.973368,0.953269,0.947368,0.971616,...,0.918008,0.926505,0.949794,0.965581,0.924690,0.986276,0.932928,0.956973,0.968983,0.962614
2022-04-11,1.033766,1.023957,1.020748,1.054862,0.968938,1.029912,1.031322,0.982491,1.012526,1.068506,...,1.032226,1.009417,1.089255,1.024262,0.989903,1.010969,0.987417,1.056080,1.041707,0.996847
2022-04-12,1.037569,1.032722,1.027071,1.038762,1.041196,1.047122,1.043612,1.018347,1.052698,1.053132,...,1.049113,1.052829,1.051526,1.036214,1.045735,1.022108,1.025716,1.055225,1.043894,1.029611
2022-04-13,0.954375,0.969587,0.958374,0.958401,0.963090,0.953312,0.966765,0.967026,0.950645,0.965795,...,0.940664,0.966600,0.965490,0.958145,0.949404,0.993339,0.956574,0.976904,0.966675,0.977887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27,1.004918,1.009861,1.004556,1.004018,1.012246,1.005777,1.009506,1.006083,1.010550,1.010861,...,1.003616,1.012157,1.007506,1.008111,1.007200,1.013398,1.005454,1.013427,1.009185,1.011281
2023-08-30,0.983450,0.977544,0.992218,0.975532,0.978364,0.976021,0.979056,0.994369,0.975965,0.966550,...,0.981170,0.972315,0.967730,0.987403,0.983695,0.978522,0.992746,0.965957,0.976760,0.984678
2023-08-31,1.008346,1.004203,1.005262,1.010809,1.007796,1.011006,1.006391,1.008804,1.006670,1.006240,...,1.010250,1.005630,1.006139,1.001784,1.009983,1.005186,1.007599,1.002376,1.006477,1.003860


In [60]:
means = outcomes.mean()
stds = outcomes.std()
ratios = means / stds

In [61]:
markowitz = pd.DataFrame({'mean':means, 'std':stds, 'sharpe':ratios})

In [62]:
markowitz.hvplot.scatter(y='mean',x='std',c='sharpe', cmap='RdYlGn', colorbar=True, size=10, width=500, height=500)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-1k2_Q7Ln-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Scatter   [std]   (mean,sharpe)

In [63]:
ratios.argmax()

290

In [64]:
portfolios.iloc[923]

token_id
bright-token                 0.129588
gitcoin                      0.125463
giveth                       0.088729
honey-finance                0.240768
metagame                     0.234508
rad                          0.161140
token-engineering-commons    0.019804
Name: 923, dtype: float64